# Cartola Clubs
Esse notebook tem como objetivo popular uma base de dados com todos os clubes de futebol que participaram no Campeonato Brasileiro série A do ano corrente.

### Dependências
Instalação de dependências necessárias

In [1]:
%pip install requests
%pip install psycopg2-binary
%pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Bibliotecas necessárias 
Importação das bibliotecas necessárias para a execução do notebook

In [2]:
import timeit
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

### Inicio
Busca dos dados na API do Cartola

In [3]:
start = timeit.default_timer()

In [4]:
clubs_page = requests.get('https://api.cartolafc.globo.com/partidas')
clubs_json = json.loads(clubs_page.content)

In [5]:
CHAMPIONSHIP_CLUBS = 20
clubs_dict = clubs_json['clubes']
if (len(clubs_dict) != CHAMPIONSHIP_CLUBS):
    raise Exception('wrong_number','O campeonato deveria ter 20 times!')

In [6]:
clubs = []
for id in clubs_dict:
    club = {'id': clubs_dict[id]['id'],
            'name': clubs_dict[id]['nome'],
            'initials': clubs_dict[id]['abreviacao']
           }
    clubs.append(club)

In [7]:
df_clubs = pd.DataFrame(clubs)
df_clubs.shape

(20, 3)

### Manutenção do Banco de dados
Nessa seção do notebook os dados do banco da tabela club serão destruídos e os novos dados serão inseridos.

**Preencha os dados da conectividade:**

In [8]:
user = 'postgres'
password = 'postgres'
address = '172.18.0.2'
db_name = 'cartola'

In [9]:
def create_database_session():
    engine = create_engine(f'postgresql://{user}:{password}@{address}/{db_name}')
    return engine

In [10]:
def erase_data(engine):
    engine.execute('TRUNCATE club')

In [11]:
def create_table(engine):
    engine.execute('''
        CREATE TABLE IF NOT EXISTS club (
            id smallint, 
            name varchar(80),
            initials varchar(3)
        )'''
    )

In [12]:
def save_data(engine, dataframe):
    dataframe.to_sql('club', con=engine, index=False, if_exists='replace')

In [13]:
engine = create_database_session()
create_table(engine)
erase_data(engine)
save_data(engine, df_clubs)

In [14]:
end = timeit.default_timer()
print ('Duração: %.f segundo(os)' % ((end - start) ))

Duração: 1 segundos
